In [15]:
import pandas as pd
import numpy as np
import datetime

#import function to query mysql server
import Functions.datamart_query as q

#set region to create territories for
region = 'UNITED_KINGDOM'

#pull in account data from SQL file based on region
with open(f'/Users/jnichols/Python Files/Territory Script 2022/Queries/{region}.sql', 'r') as myfile:
    query=myfile.read().replace('\n', ' ')
# Store query results in accounts df.
accounts = q.run_query(query, database='Production')

#read in csv file containing names and regions of reps that need territories
filename = f'Reps/{region}_REPS.xlsm'
reps = pd.read_excel(filename)

#Create and apply segmentation rules to list of accounts
def segmentation(row):
    if row['L12MSpend'] > 200000 or row['MaxRev'] > 500 or row['XMScore'] >= 200:
        return 1
    if row['L12MSpend'] > 75000 or row['MaxRev'] > 300 or row['XMScore'] >= 75:
        return 2
    if row['L12MSpend'] > 0 or row['MaxRev'] > 25:
        return 3
    else:
        return 4
    
accounts["Segment"] = accounts.apply(segmentation, axis=1)

#Create "customer" column 
def customer(row):
    if row['L12MSpend'] > 0:
        return 1
    else:
        return 0

accounts['Customer'] = accounts.apply(customer, axis=1) 


#Set territory sizes for each tier 
tier_6_territory_size = 150
tier_5_territory_size = 150
tier_4_territory_size = 200
tier_3_territory_size = 200
tier_2_territory_size = 200 

t_sizes = [tier_6_territory_size,tier_5_territory_size,tier_4_territory_size,tier_3_territory_size,tier_2_territory_size]

territory_sizes = pd.DataFrame(t_sizes, index = ["Tier 6","Tier 5","Tier 4","Tier 3","Tier 2"], columns = ["Territory Size"])

#Create variables containing number of accounts in each segment
seg_1_acc = accounts[accounts['Segment'] == 1].shape[0]
seg_2_acc = accounts[accounts['Segment'] == 2].shape[0]
seg_3_acc = accounts[accounts['Segment'] == 3].shape[0]
seg_4_acc = accounts[accounts['Segment'] == 4].shape[0]
print(seg_1_acc)

#Create variables containing how many reps of each tier we would like to create territories for
tier_6_reps = reps[reps["Tier"]==6].shape[0]
tier_5_reps = reps[reps["Tier"]==5].shape[0]
tier_4_reps = reps[reps["Tier"]==4].shape[0]
tier_3_reps = reps[reps["Tier"]==3].shape[0]
tier_2_reps = reps[reps["Tier"]==2].shape[0] 
total_reps = reps.shape[0]

r = [tier_6_reps,tier_5_reps,tier_4_reps,tier_3_reps,tier_2_reps] 

reps_in_tier = pd.DataFrame(r, index = ["Tier 6","Tier 5","Tier 4","Tier 3","Tier 2"], columns = ["Reps in Tier"])

#Count how many TBH reps we have in each tier 
tier_6_tbh = reps[(reps["Tier"]==6) & (reps["Id"].isnull())].shape[0]
tier_5_tbh = reps[(reps["Tier"]==5) & (reps["Id"].isnull())].shape[0]
tier_4_tbh = reps[(reps["Tier"]==4) & (reps["Id"].isnull())].shape[0]
tier_3_tbh = reps[(reps["Tier"]==3) & (reps["Id"].isnull())].shape[0]
tier_2_tbh = reps[(reps["Tier"]==2) & (reps["Id"].isnull())].shape[0]

#Calculate the number of accounts needed to create territories at each tier
tier_6_acc = tier_6_reps * tier_6_territory_size
tier_5_acc = tier_5_reps * tier_5_territory_size
tier_4_acc = tier_4_reps * tier_4_territory_size
tier_3_acc = tier_3_reps * tier_3_territory_size
tier_2_acc = tier_2_reps * tier_2_territory_size


account_tiers = [tier_6_acc,tier_5_acc,tier_4_acc, tier_3_acc, tier_2_acc]





/Users/jnichols/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


889


In [16]:
print(territory_sizes)
print(reps_in_tier)

        Territory Size
Tier 6             150
Tier 5             150
Tier 4             200
Tier 3             200
Tier 2             200
        Reps in Tier
Tier 6             2
Tier 5             3
Tier 4             4
Tier 3             4
Tier 2             0


In [17]:
#The following formula round-robins accounts within a single segment to each tier according to a weighted ratio
def seg_round_robin(seg, account_tiers):
    
    i=0
    t6=0
    t5=0
    t4=0
    t3=0
    t2=0
    
    while i < seg:
        if t6 + 5 <= account_tiers[0]:
            t6 += 5 
            i += 5
        if t5 + 4 <= account_tiers[1]: 
            t5 += 4
            i += 4
        if t4 + 3 <= account_tiers[2]: 
            t4 += 3 
            i += 3
        if t3 + 2 <= account_tiers[3]: 
            t3+= 2 
            i += 2
        if t2 + 1 <= account_tiers[4]:
            t2 += 1
            i += 1
        else:
            i += 1
            
    
    account_totals = [t6,t5,t4,t3,t2]
    
    return account_totals

In [18]:
#This calculates optimal account distribution across all tiers and segments using the above formula and 
#places all of them into a single dataframe for easy reference. GLHF

#Segment 1
account_totals = seg_round_robin(seg_1_acc, account_tiers) 

for i in range(len(account_tiers)):
    account_tiers[i] -= account_totals[i]

account_distribution = pd.DataFrame([account_totals], index = ["Segment 1"], columns = ["Tier 6", "Tier 5", "Tier 4", "Tier 3", "Tier 2"])

#Segment 2
account_totals = seg_round_robin(seg_2_acc, account_tiers)

x = pd.DataFrame([account_totals], index = ["Segment 2"], columns = ["Tier 6", "Tier 5", "Tier 4", "Tier 3", "Tier 2"])

account_distribution = pd.concat([account_distribution,x])
                                 
for i in range(len(account_tiers)):
    account_tiers[i] -= account_totals[i]

#Segment 3
account_totals = seg_round_robin(seg_3_acc, account_tiers)

x = pd.DataFrame([account_totals], index = ["Segment 3"], columns = ["Tier 6", "Tier 5", "Tier 4", "Tier 3", "Tier 2"])

account_distribution = pd.concat([account_distribution,x])

for i in range(len(account_tiers)):
    account_tiers[i] -= account_totals[i]

#Segment 4
account_totals = seg_round_robin(seg_4_acc, account_tiers)

x = pd.DataFrame([account_totals], index = ["Segment 4"], columns = ["Tier 6", "Tier 5", "Tier 4", "Tier 3", "Tier 2"])

account_distribution = pd.concat([account_distribution,x])

print(account_distribution) 


           Tier 6  Tier 5  Tier 4  Tier 3  Tier 2
Segment 1     300     240     180     120       0
Segment 2       0     208     348     232       0
Segment 3       0       0     270     448       0
Segment 4       0       0       0       0       0


In [19]:
#Assign accounts with open opps or positive spend to their previous owners to minimize disruption,
#filtering to ensure that the rep's name appears on the rep sheet (for obvious reasons old/outdated reps do not receive their accounts)

current_priority_accounts = accounts[['DeploymentAccountID','OwnerUserName','OwnerUserID']][(accounts["OpenOpps"] > 0) | (accounts["Customer"] == 1)]
current_priority_accounts = pd.merge(current_priority_accounts, reps, how = 'inner', left_on = 'OwnerUserID', right_on = 'Id')
current_priority_accounts = current_priority_accounts[['DeploymentAccountID', 'OwnerUserName', 'OwnerUserID']]
current_priority_accounts.columns = ['DeploymentAccountID', 'NewOwnerUserName', 'NewOwnerUserID']
accounts = pd.merge(accounts, current_priority_accounts, how = 'left', on = 'DeploymentAccountID')


In [20]:
def territory_engine(r, a, ad, rt):
    available_accounts = a[(a["NewOwnerUserID"].isnull())]
    assignments = a
    assigned_accounts = pd.DataFrame()
    
    for i, row in r.iterrows():
        owned_accounts = a[(a["NewOwnerUserID"] == row["Id"])]
        name = row["Name"]
        id = row["Id"]
        tier = row["Tier"] 
        tier_str = f"Tier {tier}"
        reps_in_rep_tier = rt.loc[tier_str,"Reps in Tier"]
        dist = ad[tier_str]
        dist = dist.apply(lambda x: int(np.floor(x / reps_in_rep_tier)))
        print(f'*** {name} ***')
        print(f'Tier: {tier}')
        for i in range(len(dist)):
            print(f"* Segment {i+1} *")
            owned_seg_accounts = owned_accounts[(owned_accounts["Segment"]) == i + 1].shape[0]
            print(f"Has: {owned_seg_accounts}")
            required_seg_accounts = dist[i] - owned_seg_accounts
            print(f"Needs: {required_seg_accounts}")
            if required_seg_accounts < 0:
                required_seg_accounts = 0
            available_in_seg = available_accounts[(available_accounts["Segment"]) == i + 1]
            print(f"Available: {available_in_seg.shape[0]}")
            assigned_seg_accounts = available_in_seg.sample(n=required_seg_accounts)
            assigned_seg_accounts = assigned_seg_accounts.assign(NewOwnerUserName = name, NewOwnerUserID = id)
            print(f"Assigning: {assigned_seg_accounts.shape[0]}")
            assigned_accounts = pd.concat([assigned_accounts, assigned_seg_accounts])
            available_accounts.update(assigned_seg_accounts)
            x = available_accounts[(available_accounts["NewOwnerUserID"].isnull())]
            available_accounts = x
            print(f"Remaining available: {available_accounts.shape[0]}")
            available_accounts.reset_index(inplace = True, drop = True)
            assigned_accounts.to_csv('Output/assigned_checkpoint.csv', index = False)
        
    assigned_accounts.to_csv('Output/assigned_accounts_out.csv', index = False)
    assigned_accounts.reset_index(inplace = True, drop = True)
    assignments.update(assigned_accounts)
    assignments.to_csv('Output/accounts_out.csv', index = False)
            

In [21]:
territory_engine(reps, accounts, account_distribution, reps_in_tier)

*** Lorna Bennett ***
Tier: 3
* Segment 1 *
Has: 0
Needs: 30
Available: 814
Assigning: 30
Remaining available: 45963
* Segment 2 *
Has: 0
Needs: 58
Available: 862
Assigning: 58


/var/folders/ms/09qqsxrs0650n24l0sr2vqy40000gp/T/ipykernel_53724/3304264362.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  available_accounts.update(assigned_seg_accounts)
/var/folders/ms/09qqsxrs0650n24l0sr2vqy40000gp/T/ipykernel_53724/3304264362.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  available_accounts.update(assigned_seg_accounts)


Remaining available: 45905
* Segment 3 *
Has: 0
Needs: 112
Available: 14748
Assigning: 112
Remaining available: 45793
* Segment 4 *
Has: 0
Needs: 0
Available: 29569
Assigning: 0
Remaining available: 45793
*** Michael Lamb ***
Tier: 4
* Segment 1 *
Has: 0
Needs: 45
Available: 784
Assigning: 45
Remaining available: 45748
* Segment 2 *
Has: 0
Needs: 87
Available: 804
Assigning: 87


/var/folders/ms/09qqsxrs0650n24l0sr2vqy40000gp/T/ipykernel_53724/3304264362.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  available_accounts.update(assigned_seg_accounts)


Remaining available: 45661
* Segment 3 *
Has: 0
Needs: 67
Available: 14636
Assigning: 67
Remaining available: 45594
* Segment 4 *
Has: 0
Needs: 0
Available: 29569
Assigning: 0
Remaining available: 45594
*** Robin Matthews ***
Tier: 4
* Segment 1 *
Has: 13
Needs: 32
Available: 739
Assigning: 32
Remaining available: 45562
* Segment 2 *
Has: 10
Needs: 77
Available: 717
Assigning: 77
Remaining available: 45485
* Segment 3 *
Has: 3
Needs: 64
Available: 14569
Assigning: 64
Remaining available: 45421
* Segment 4 *
Has: 1
Needs: -1
Available: 29569
Assigning: 0
Remaining available: 45421
*** Alan McHugh ***
Tier: 5
* Segment 1 *
Has: 11
Needs: 69
Available: 707
Assigning: 69
Remaining available: 45352
* Segment 2 *
Has: 4
Needs: 65
Available: 640
Assigning: 65
Remaining available: 45287
* Segment 3 *
Has: 7
Needs: -7
Available: 14505
Assigning: 0
Remaining available: 45287
* Segment 4 *
Has: 0
Needs: 0
Available: 29569
Assigning: 0
Remaining available: 45287
*** Fionn Murphy ***
Tier: 4
* Segm

In [22]:
accounts.to_csv('Output/raw_accounts_out.csv', index = False)